# Decorators and Context Managers
Decorators are a feature in Python that allows you to add or modify the behavior of functions without changing their original code. They are commonly used for tasks such as logging, validation, caching, or measuring execution time. Decorators work by wrapping one function inside another and returning the modified version.

Context managers are used to manage external resources such as files, database connections, or network sessions. By using the with statement, a context manager ensures that resources are properly opened and closed, even if an error occurs. This prevents resource leaks and helps keep code clean and safe.

# Simple


## Example 1: Executing a Timing Function with decorator

This decorator measures and prints the execution time of the wrapped function. It's suitable for assessing function performance.

In [1]:
import time

def timing_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Function {func.__name__} took {end_time - start_time} seconds to execute.")
        return result
    return wrapper

@timing_decorator
def sample_function():
    return 500 ** 2

sample_function()


Function sample_function took 4.76837158203125e-07 seconds to execute.


250000

## Example 2: Logging Function Execution with decorator

This decorator prints the arguments passed to the function and the returned results. Useful for debugging and function tracing.

In [2]:
def log_decorator(func):
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with args: {args}, kwargs: {kwargs}")
        result = func(*args, **kwargs)
        print(f"{func.__name__} returned {result}")
        return result
    return wrapper

@log_decorator
def add(a, b):
    return a + b

results = add(2, 3)


Calling add with args: (2, 3), kwargs: {}
add returned 5


In [3]:
results

5

# Intermediate

## Example 1: Timing Code Blocks with a Context Manager

This context manager measures the time taken by a block of code inside with. It helps monitor the performance of a specific part of the program.

In [4]:
import time
from contextlib import contextmanager

@contextmanager
def timer_context_manager():
    start_time = time.time()
    yield
    end_time = time.time()
    print(f"Code block took {end_time - start_time} seconds to run.")

with timer_context_manager():
    time.sleep(3)


Code block took 3.0002999305725098 seconds to run.


## Example 2: Custom Context Manager for Resource Management

This context manager handles the process of opening and closing files automatically, ensuring that files are always closed even if errors occur.

In [5]:
from contextlib import contextmanager

@contextmanager
def file_opener(filename, mode):
    try:
        file = open(filename, mode)
        yield file
    finally:
        file.close()

with file_opener("sample.txt", "w") as file:
    file.write("Hello, Context Managers!")


# Advanced

## Example 1: Advanced Logging Decorator with Signature Function

This decorator prints the function name, full arguments (both positional and keyword), and the return result. A more informative version of logging.

In [6]:
def advanced_log_decorator(func):
    def wrapper(*args, **kwargs):
        args_str = ', '.join([str(arg) for arg in args])
        kwargs_str = ', '.join([f"{key}={value}" for key, value in kwargs.items()])
        all_args = ', '.join(filter(None, [args_str, kwargs_str]))

        result = func(*args, **kwargs)
        print(f"Function {func.__name__}({all_args}) returned {result}")
        return result
    return wrapper

@advanced_log_decorator
def multiply(x, y):
    return x * y

multiply(4, 5)


Function multiply(4, 5) returned 20


20

## Example 2: Advanced Context Managers for Exception Handling

This context manager catches certain exceptions during the execution of the with block and prints an error message. This helps with more elegant error handling.

In [7]:
from contextlib import contextmanager

@contextmanager
def exception_handler(exception_type):
    try:
        yield
    except exception_type as e:
        print(f"Caught exception: {e}")

with exception_handler(ZeroDivisionError):
    result = 10 / 0


Caught exception: division by zero


### Logging in Web Applications (Flask)

Case Study: Monitoring API requests for debugging and analysis.

Every API request to /home will be logged with the time and method.

Can be used across multiple API endpoints without rewriting logging code.

In [8]:
from flask import Flask, request
import datetime

app = Flask(__name__)

# Decorator for logging requests
def request_logger(func):
    def wrapper(*args, **kwargs):
        print(f"[{datetime.datetime.now()}] Request to {request.path} with {request.method}")
        return func(*args, **kwargs)
    return wrapper

@app.route("/home", methods=["GET"])
@request_logger
def home():
    return "Welcome to the Home Page"

if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\LENOVO\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Caching with Decorators

Case: Speed up API responses by caching calculation results.

If a function has already been executed, the results are cached.

Speeds up applications, especially when calculations are expensive (e.g., database queries or image processing).

In [9]:
import functools

cache = {}

def caching_decorator(func):
    @functools.wraps(func)
    def wrapper(*args):
        if args in cache:
            print("Fetching from cache...")
            return cache[args]
        result = func(*args)
        cache[args] = result
        return result
    return wrapper

@caching_decorator
def expensive_computation(x):
    print("Computing...")
    return x ** 2  # expensive computation simulation

print(expensive_computation(5)) # First time: Counting
print(expensive_computation(5))  # Second time: Fetch from cache


Computing...
25
Fetching from cache...
25


In [10]:
print(expensive_computation(10))
print(expensive_computation(2))

Computing...
100
Computing...
4


### Database Connection Management (SQLAlchemy)

Case: Ensuring that database connections are closed after use.

The context manager ensures that database connections are always closed after use.

Prevents database connection leaks, which can cause system crashes.

In [12]:
from sqlalchemy import create_engine, text  # Add 'text'
from sqlalchemy.orm import sessionmaker
from contextlib import contextmanager

# Initialize connection to SQLite database
DATABASE_URL = "sqlite:///example.db"
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(bind=engine)

# Context manager for database sessions
@contextmanager
def get_db_session():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()  # Make sure the connection is always closed

# Use sessions to run queries
with get_db_session() as session:
    result = session.execute(text("SELECT sqlite_version();"))  # Wrap the query with text()
    print(result.fetchall())


[('3.45.3',)]


### Managing API Connections with Requests

Case: Retrieving data from an external API using a secure connection.

The context manager ensures the API connection is closed after use.

Avoid HTTP session leaks, which can waste server resources.

In [13]:
import requests
from contextlib import contextmanager

@contextmanager
def open_api_session(url):
    session = requests.Session()
    try:
        yield session.get(url)  # Returns API response
    finally:
        session.close()  # Close the session after completion

API_URL = "https://jsonplaceholder.typicode.com/todos/1"

with open_api_session(API_URL) as response:
    print(response.json())  # Retrieving data from API

{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False}
